#Prepare environment

In [1]:
%matplotlib inline

In [2]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #pour faire des graph dans la bonne couleur etc.. par default
import glob
import re
import csv as csv
import random as rn
import math
import ast #http://stackoverflow.com/questions/1894269/convert-string-representation-of-list-to-list-in-python
#import to_datetime
import datetime
from ggplot import *
from __future__ import division

#from lxml.html import parse
from urllib2 import urlopen
import webbrowser

#Upload datasets

> Define products to be taken into account for credited conversions

>/!\ Which products shall be taken into account?

In [3]:
#http://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe
gamme_prod = ['PD','DI','DN','TI','TN','AM','PR','GC','Mixte']


cpm = 4.0

> Name of the files to be uploaded

> These should be hosted in the root folder

In [4]:
#file_quanti, 3PAS equivalent (Excel spreadsheet) 
adServerFile = 'Report_Quantcast_mobilite_Standard_v1_M88Z5cUHryFJpRP_AA1S.xlsx'
#file_quali 
detailedFile = 'Report_Quantcast_mobilite_conversion_view_PNJjl0461vpIzpMgHrA7.csv'

In [5]:
#first file to be uploaded details of products part of each conversion. 
#We'll need to treat and refine this file so that only relevant products are being shown.
detailedConversions = pd.read_csv(detailedFile, dtype=object)

#We need to update the date format and strip hours
#http://stackoverflow.com/questions/26387986/strip-time-from-an-object-date-in-pandas
detailedConversions['conversion_date'] = pd.to_datetime(detailedConversions['Conversion date-hour'])
detailedConversions['conversion_date'] = detailedConversions['conversion_date'].apply(lambda x:x.date().strftime('%Y-%m-%d'))

#detailedConversions details products within each orders
#check what's in detailedConversions
#detailedConversions.count()

In [6]:
#then upload the second file. this is the regular 3PAS files with every conversions to which we contributed. 
xls_file = pd.ExcelFile(adServerFile)
adServerTable = xls_file.parse('DataView')

#adServerTable credits each order to a vendor
#check what's in adServerTable
#adServerTable.count()

#Create matching key on both datasets

In [7]:
#create function that will be used to create matching key by concatenating several columns
#http://stackoverflow.com/questions/23444858/concatenate-column-values-in-pandas-dataframe-with-nan-values
def concat(*args):
    strs = [str(arg) for arg in args if not pd.isnull(arg)]
    return '_'.join(strs) if strs else np.nan

np_concat = np.vectorize(concat)

In [8]:
#We need to define the key to join both file. 
#first 3PAS report Key - file_quanti
adServerTable['key'] = np_concat(adServerTable['Date'], adServerTable['Site/Offer ID'], adServerTable['Insertion ID'], adServerTable['Creative ID'])
#QA the created key
#adServerTable[['Date','Site/Offer ID','Insertion ID','Creative ID', 'key']].head(2)

In [9]:
#Then file_quali
detailedConversions['key'] = np_concat(detailedConversions['conversion_date'], detailedConversions['Site-Offer ID'], detailedConversions['Insertion ID'], detailedConversions['Creative ID'])
#QA the created key
#detailedConversions[['conversion_date','Site-Offer ID','Insertion ID','Creative ID', 'key']].head(2)

> Both data sets have now corresponding matching key.

#Define functions for latter use

> Let's define several functions that'll be used for cleaning datasets

In [10]:
# function aimed at cleaning revenue figures

def cleanrev(col):
    if col == '-':
        return 0.0
    else:
        return float(col)

In [11]:
# function aimed at cleaning products part of the orders. 
#Distinguish monoproducts orders from ones with multiple products (ones with relevant products or non relevant product - as per the campaign)

def cleanGamme(col,gamme_prod):
    items = list(set(col.split("|")) )
    if len(items) == 1:
        return items[0]
    elif re.search('|'.join(gamme_prod),col):
        return "Mixte"
    else:
        return "Mixte_non_comptabilise"
        

In [12]:
# function aimed at associating products and unit price
# /!\ We need to verify every product are added and quantity matches

def cleanProduitMix(df):
    items = df[0].split("|")
    prices = df[1].split("|")
    quantity = df[2].split("|")
    if len(items) == 1:
        return items[0] + '-%s-%s' %(float(prices[0]), float(quantity[0]))
    else:
        l = len(items)
        mix = ''
        for i in range(0,len(items)-2):
            mix = mix + items[i] + '-%s-%s|' %(float(prices[i]), float(quantity[i]))
        return mix + items[l-1] + '-%s-%s' %(float(prices[l-1]), float(quantity[l-1]))

#define the same function without tacking into account the quantity section    
def cleanProduitMixWOQuantities(df):
    items = df[0].split("|")
    prices = df[1].split("|")
    quantity = df[2].split("|")
    if len(items) == 1:
        return items[0] + '-%s' %(float(prices[0]))
    else:
        l = len(items)
        mix = ''
        for i in range(0,len(items)-2):
            mix = mix + items[i] + '-%s|' %(float(prices[i]))
        return mix + items[l-1] + '-%s' %(float(prices[l-1]))

In [13]:
#function aimed at concatenating several columns separated by '-'
def concatenate(df):
    if len(df)>0:
        c = ''
        l = len(df)-1
        for i in range (0, l-1):
            c = c + df[i] + '-'
        return c + df[l]
    else:
        return ''

In [14]:
#function to be used when filtering out products not in required
def filter_products(x):
    if re.search('|'.join(gamme_prod),x):
        return "true"
    else:
        return "false"

#Treat and refine qualitative file

> The goal here is to:

>> remove products not relevant for our media campign

>>agregate orders constituted by credited products to the 3PAS file

In [15]:
#select only relevant columns. remove NA.
#you could use detailedConversions.count() to check what columns are part of the dataset 
details3PAS = detailedConversions[['key','nom_site','gamme_produit', 'prix_produit','nombre_produit' ,'prix_ht_zanox']]
details3PAS = pd.DataFrame(details3PAS).dropna()

#then clean revenue and order categories
details3PAS['revenue_transaction'] = details3PAS.prix_ht_zanox.apply(lambda x: cleanrev(x))
details3PAS['panier'] = details3PAS.gamme_produit.apply(lambda x: cleanGamme(x,gamme_prod))

#details3PAS.head(4)
#check the result for a given order ID (key)
#details3PAS[details3PAS.key == '2016-04-08_1123_7485_1731'].head()

In [16]:
#first step with this qualitative dataset to count relevant conversions.
#We need this first since we'll revamp the dataset to take into account only relevant revenu
dataSet3PAS= details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
dataSet3PAS['selected'] = dataSet3PAS['panier'].apply(lambda x: filter_products(x))
dataSet3PAS = dataSet3PAS[dataSet3PAS['selected'] == 'true']

#get count of conversions relevant to us, this will be used later on when we'll aggregate datasets.
transactions = dataSet3PAS.groupby(['key', 'nom_site']).agg({
                                     'panier': 'count',
                                     'revenue_transaction': sum
                                               }).reset_index()    

#rename columns and check results.
transactions.columns = ['key', 'nom_site', 'relevant_conversions', 'total_revenue']
#transactions.head(3)


#transactions = dataSet3PAS.groupby(['key']).count().reset_index()
#revenue = dataSet3PAS.groupby(['key']).sum().reset_index()
#transactions.head(10)
#revenue.head(10)
#ds = transactions.join(revenue).reset_index()
#ds.columns = ['key', 'revenue', 'conversions']
#ds.head(5)

### /!\ update cell below based on the structure of the dataset

In [17]:
#continue with the qualitative dataset, remove unnescessaries variables and apply function to filter out products
#reorder products with unit prices
#http://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe
#http://stackoverflow.com/questions/19914937/applying-function-with-multiple-arguments-to-create-a-new-pandas-column

## depending on the structure of the dataset use cleanProduitMix or cleanProduitMixWOQuantities (quantities being taken into account in the rveenue)
#details3PAS['order'] = details3PAS[['gamme_produit','prix_produit','nombre_produit']].apply(cleanProduitMix, axis = 1)
details3PAS['order'] = details3PAS[['gamme_produit','prix_produit','nombre_produit']].apply(cleanProduitMixWOQuantities, axis = 1)

details3PAS = details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)

#details3PAS.head(3)

> Revenues don't always exactly match. 

> This might not be material (probably due to long strings stripped or VAT or shipping) but could worth checking with the customer.

> We now need to create a row per product purchased.

In [18]:
#http://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-rows
#http://stackoverflow.com/questions/33622470/fast-way-to-split-column-into-multiple-rows-in-pandas

#create a column with only porduct and unit prices
#we'll use stack function
s = details3PAS['order'].str.split("|", expand=True).stack() #.reset_index()
i = s.index.get_level_values(0)
df = details3PAS.loc[i].copy()
df["order"] = s.values
#df.head(20)

### /!\ update cell below based on the structure of the dataset

In [19]:
df_bis = df 

#make two columns out of this. might be tricky if we want to keep the original DF 
##Update based on the structure of the dataset use cleanProduitMix or cleanProduitMixWOQuantities
#if cleanProduitMix function used
#df_bis["product"], df_bis["price"], df_bis["quantity"] = zip(*df_bis["order"].str.split('-',2))
#if cleanProduitMixWOQuantities function used
df_bis["product"], df_bis["price"] = zip(*df_bis["order"].str.split('-',1))


df_bis["product"] = df_bis["product"].apply(lambda x: str(x))
df_bis["price"] = df_bis["price"].apply(lambda x: float(x))

##uncomment if quantity is being taken into account
#df_bis["quantity"] = df_bis["quantity"].apply(lambda x: float(x))


#df_bis[df_bis.key == '2016-03-14_1123_7491_1740'].head()

### /!\ update cell below based on the structure of the dataset

In [20]:
#compute revenue for each product sold
#uncomment the relevant line whether you take into account the quantity

#df_bis['revenue_product'] = df_bis['price'] * df_bis['quantity']
df_bis['revenue_product'] = df_bis['price'] * 1


df_bis = df_bis[['key', 'nom_site', 'product', 'revenue_product']]
#df_bis[df_bis.key == '2016-03-14_1123_7491_1740'].head()

In [21]:
#keep only products concerned by our advertising campaign (products part of gamme_prod)

df_final = df_bis
#using our function defined above
df_final["product_bis"] = df_final["product"].apply(lambda x: filter_products(x))

#filtering out false products (products not concerned by the campaign)
df_final = df_final[df_final['product_bis'] == "true"]
#df_final.head()

/private/var/localadmin/.conda/envs/py27/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
# Let's sum valid revenue per transaction key (order ID, one revenue per order)
# Group the data frame by month and item and extract a number of stats from each group

df_final_bis = df_final.groupby(['key', 'nom_site']).agg({
                                     'revenue_product':sum
                                               }).reset_index()    

#Add the total number conversions we stored in transactions DataFrame. We'll merge the two datasets.
#transactions.columns = ['key', 'nom_site', 'relevant_conversions', 'total_revenue']
#df_final_cleaned = pd.concat([df_final_bis,transactions], keys=['key'], join= 'inner' ,ignore_index=True)
#df_final_cleaned = df_final_bis.append(transactions)
df_final_cleaned = pd.merge(df_final_bis,transactions, how= 'inner', on = ['key', 'nom_site'] ,left_index=False,right_index=False)

#df_final_cleaned.head(3)

> Performances summary

# /!\ filter dataset - first option not considered anymore since we computed revenue per product per transaction key

> You may jump to the following section.

In [23]:
#details3PAS.count()
#details3PAS.head(4)

In [25]:
#dataSet3PAS= details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
#dataSet3PAS.head(3)

In [24]:
#dataSet3PAS = dataSet3PAS[dataSet3PAS.panier.isin(gamme_prod)]
#dataSet3PAS.head(5)

In [25]:
#transactions = dataSet3PAS.groupby(['key']).count()
#revenue = dataSet3PAS.groupby(['key']).sum()
#ds = pd.DataFrame(transactions,revenue)

In [26]:
#transactions = pd.DataFrame(dataSet3PAS.groupby(["key"]).sum()["revenue_transaction"])
#revenue = pd.DataFrame(dataSet3PAS.groupby(["key"]).count()["panier"])
#ds = transactions.join(revenue).reset_index()
#c = np.append('key', 'revenue', 'conversions',axis =0)
#ds.columns = ['key', 'revenue', 'conversions']
#ds.head(5)

In [27]:
#dataSet3PAS[dataSet3PAS.key == '2016-03-10_1123_7485_1731']

In [28]:
#ds[ds.key == '2016-03-10_1123_7485_1731']

# Merge quantitative 3PAS report and qualitative report

> load quantitative 3PAS report and merge two files

In [29]:
#final_ds = pd.merge(adServerTable,ds, on='key')
final_ds = pd.merge(adServerTable,df_final_cleaned, on='key')
#final_ds[final_ds.key == '2016-03-10_1123_7485_1731'].head()

In [30]:
#keep only relevant columns for reporting purpose
final_ds = final_ds[['Date', 'Site/Offer ID','Site/Offer', 'Insertion ID', 'Insertion', 'Creative ID', 'Creative', 'Creative Sizes','Imp.', 'Clicks','relevant_conversions', 'revenue_product','total_revenue', 'key' ]]
#final_ds.head(3)

In [32]:
#add a few additional metrics such as cpa and roi
#we first need the advertsier cost based on the cpm sold
final_ds['cost'] = cpm * final_ds['Imp.'] / 1000
#we can now compute cpa and ROI
final_ds['cpa'] = final_ds['cost'] / final_ds['relevant_conversions']
final_ds['roi'] = final_ds['revenue_product'] / final_ds['cost']

#should you want to double check the results
#final_ds.head()

# Generate a CSV file that needs to be sent to campaign analyst

In [33]:
final_ds.to_csv('3pas_report_final.csv', index=False,encoding='utf-8')
print('Done')

Done
